In [1]:
# Install Java, Spark, and Findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-04 18:08:35--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.75MB/s    in 0.2s    

2020-08-04 18:08:36 (4.75 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AWSVineReview").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Configure settings for RDS data-bootcamp.crrqs5jt3ck3.us-east-1.rds.amazonaws.com
jdbc_url="jdbc:postgresql://data-bootcamp.crrqs5jt3ck3.us-east-1.rds.amazonaws.com:5432/bigdatahomework"
config = {"user":"xxxxxx", 
          "password": "xxxxxxx", 
          "driver":"org.postgresql.Driver"}


In [5]:
# read vine_table in 16 partitions, otherwise it will be out of memory or crash
vine_df = spark.read.jdbc(url=jdbc_url, table="vine_table", properties=config, column='id', lowerBound=1, upperBound =7692330, numPartitions=16)
vine_df.show()

+--------------+-----------+-------------+-----------+----+---+
|     review_id|star_rating|helpful_votes|total_votes|vine| id|
+--------------+-----------+-------------+-----------+----+---+
| RN7NUCV877DPM|          5|            0|          0|   N|  1|
|R368YD4DSU7PIS|          2|            2|          3|   N|  2|
|R1MB5R3EBZE4SL|          4|            0|          0|   N|  3|
| RE82J6O1PYR10|          5|            1|          2|   N|  4|
|R3L1UKVV3STV96|          5|            0|          0|   N|  5|
|R24PX4RW07VMMF|          3|            0|          0|   N|  6|
|R2LVU2ITIMLZ3M|          4|            0|          0|   N|  7|
|R1XCTVX8MV1F6F|          5|            0|          0|   N|  8|
| REZVPH99RJ26S|          5|            0|          0|   N|  9|
|R312EFXU9HGA7D|          4|            0|          0|   N| 10|
|R1O9RUGSETAFW3|          5|            0|          0|   N| 11|
|R3754RW1XIN3H6|          1|            0|          0|   N| 12|
|R3I9JJVJ7TFO49|          5|            

In [8]:
# remove null rows
vine_df = vine_df.dropna()

In [9]:
# show count
groupby_vine_df = vine_df.groupBy("vine").count()
groupby_vine_df.show()

+----+-------+
|vine|  count|
+----+-------+
|   Y|   6627|
|   N|7685692|
+----+-------+



In [10]:
# show average
grouby_avg_df = vine_df.groupBy("vine").avg()
grouby_avg_df.show()

+----+------------------+------------------+------------------+------------------+
|vine|  avg(star_rating)|avg(helpful_votes)|  avg(total_votes)|           avg(id)|
+----+------------------+------------------+------------------+------------------+
|   Y| 4.135958955786932|2.0544741210200694| 2.750716764750264| 4454659.508978422|
|   N|4.0946672075852115|1.2719140969999838|1.7666930967309125|3845638.9271827443|
+----+------------------+------------------+------------------+------------------+



In [11]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# To reduce noice, I remove all reivews with no helpful votes.
helpful_df = vine_df.filter(col("helpful_votes") > 0)
helpful_df.show()

+--------------+-----------+-------------+-----------+----+---+
|     review_id|star_rating|helpful_votes|total_votes|vine| id|
+--------------+-----------+-------------+-----------+----+---+
|R368YD4DSU7PIS|          2|            2|          3|   N|  2|
| RE82J6O1PYR10|          5|            1|          2|   N|  4|
|R3I9JJVJ7TFO49|          5|            1|          1|   N| 13|
| RW3QXRAGF3JZK|          5|            1|          2|   N| 24|
|R1BAUPA1HV60AT|          5|            1|          1|   N| 29|
|R3PAVWNQHOBVOE|          5|            1|          1|   N| 34|
| RMNNZEPE8FD34|          5|            1|          1|   N| 44|
| R3RXDH15FV2HD|          4|            6|          6|   N| 84|
|R33AXWHFH5FHQR|          1|            1|          1|   N| 85|
|R1NF1EQQ6TQL3L|          5|            2|          2|   N| 95|
|R2ZH3WQUMNZD5F|          5|            1|          1|   N| 99|
|R24HTUQRVZ0PHL|          5|            2|          2|   N|100|
|R146AKBD8YVURO|          5|            

In [12]:
avg_df = helpful_df.groupBy("vine").avg()
avg_df.show()
# average start_rating 4.07(Y)  > 3.82(N)

+----+------------------+------------------+-----------------+------------------+
|vine|  avg(star_rating)|avg(helpful_votes)| avg(total_votes)|           avg(id)|
+----+------------------+------------------+-----------------+------------------+
|   Y| 4.076166180758017| 4.961734693877551|6.231413994169096| 4404058.358600583|
|   N|3.8273818794644066| 4.558504650583689|5.936488499213323|3954076.6893523876|
+----+------------------+------------------+-----------------+------------------+



In [13]:
# take reviews with at least one vote
total_vote_df = vine_df.filter(col("total_votes") > 0)
total_vote_df.show()

+--------------+-----------+-------------+-----------+----+---+
|     review_id|star_rating|helpful_votes|total_votes|vine| id|
+--------------+-----------+-------------+-----------+----+---+
|R368YD4DSU7PIS|          2|            2|          3|   N|  2|
| RE82J6O1PYR10|          5|            1|          2|   N|  4|
|R3I9JJVJ7TFO49|          5|            1|          1|   N| 13|
| RW3QXRAGF3JZK|          5|            1|          2|   N| 24|
|R1BAUPA1HV60AT|          5|            1|          1|   N| 29|
|R28CUD297HJLP4|          1|            0|          1|   N| 33|
|R3PAVWNQHOBVOE|          5|            1|          1|   N| 34|
| RMNNZEPE8FD34|          5|            1|          1|   N| 44|
|R3M4I70GNELQHF|          5|            0|          1|   N| 45|
| R3RXDH15FV2HD|          4|            6|          6|   N| 84|
|R33AXWHFH5FHQR|          1|            1|          1|   N| 85|
|R1NF1EQQ6TQL3L|          5|            2|          2|   N| 95|
|R2ZH3WQUMNZD5F|          5|            

In [14]:
avg_ttv_df = total_vote_df.groupBy("vine").avg()
new_df = avg_ttv_df.withColumn('hepful%', avg_ttv_df['avg(helpful_votes)']/avg_ttv_df['avg(total_votes)']*100)
new_df.show()
# average star_rating 4.03(Y) > 3.80(N)

+----+-----------------+------------------+-----------------+------------------+-----------------+
|vine| avg(star_rating)|avg(helpful_votes)| avg(total_votes)|           avg(id)|          hepful%|
+----+-----------------+------------------+-----------------+------------------+-----------------+
|   Y|4.039988492520138|3.9168584579976984|5.244246260069045|4377632.1869965475|74.68868286795765|
|   N|3.801681672084821|  3.66669329795498|5.093049721365458|3965471.0001680395|71.99406050510599|
+----+-----------------+------------------+-----------------+------------------+-----------------+



### Conclusion:
* Vine reviews in general have higher start ratings (4.03 > 3.80)
* Vine reviews have higher percentage of helpful votes. (74.7% > 72.0%)
* in short, Vine reviews are useful, but we should keep in mind that their ratings are usually higher than normal.
